# Recuperação da Informação e Busca na Web - 2018.1

### Atividade: Lab 02 - Parte 1 - Expansão de Consultas
### Aluno: Johanny de Lucena Santos

## - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

## Importações necessárias para a análise dos dados

In [1]:
import pandas as pd

import numpy as np

from scipy import sparse

import nltk
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

In [9]:
#nltk.download('stopwords')



## Definindo o arquivo com os dados a ser analisado

In [2]:
dataset = pd.read_csv('../data/estadao_noticias_eleicao.csv', encoding="utf-8")

#Substituindo linhas com valores NaN por string vazia, para não atrapalhar nas operações
dataset = dataset.replace(np.nan, '', regex=True)

## Realizando junção dos títulos, subtítulos e conteúdos
Na junção, está sendo removidos todos os acentos para facilitar a criação dos tokens dos dados

In [3]:
documentos = dataset.titulo + " " + dataset.subTitulo + " " + dataset.conteudo
documentos = documentos.fillna("")

## Matrix and Vocabulary construction

In [4]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

## Removendo pontuação

In [6]:
tokenizer = RegexpTokenizer(r'\w+')
tokens_lists = documentos.apply(lambda text: tokenizer.tokenize(text.lower()))

## Removendo stopwords

In [10]:
stopword_ = stopwords.words('portuguese')
filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])

## Transformando lista de listas em uma lista

In [11]:
tokens = [token for tokens_list in filtered_tokens for token in tokens_list]

In [12]:
matrix, vocab = co_occurrence_matrix(tokens)

## Consulta da frequência dos bigramas

In [13]:
consultable_matrix = matrix.tocsr()

In [14]:
def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

In [16]:
def top_3_words(termo):
    ranking = []
    for termo_matrix in consultable_matrix:
        ranking.append(consult_frequency(termo, termo_matrix))
    
    return sorted(ranking, reverse=True)[0:3]

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

## Exemplo consult frequency

In [18]:
w1 = 'poucos'
w2 = 'recursos'
consult_frequency(w1,w2)

3